<a href="https://colab.research.google.com/github/R0hittpal/dipendency/blob/main/Advanced_Vision_AI_Fast_Tracking_Image_Classification_with_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds

# Load the Oxford Flowers 102 dataset
try:
    dataset, info = tfds.load('oxford_flowers102:2.1.1', with_info=True, as_supervised=True)

    # Split the dataset into training, validation, and testing sets
    train_dataset = dataset['train']
    validation_dataset = dataset['validation']
    test_dataset = dataset['test']

    # Print information about the dataset
    print(info)

except Exception as e:
    print(f"Error loading dataset: {e}")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.TO4NV0_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.TO4NV0_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='oxford_flowers102',
    full_name='oxford_flowers102/2.1.1',
    description="""
    The Oxford Flowers 102 dataset is a consistent of 102 flower categories commonly
    occurring in the United Kingdom. Each class consists of between 40 and 258
    images. The images have large scale, pose and light variations. In addition,
    there are categories that have large variations within the category and several
    very similar categories.
    
    The dataset is divided into a training set, a validation set and a test set. The
    training set and validation set each consist of 10 images per class (totalling
    1020 images each). The test set consists of the remaining 6149 images (minimum
    20 per class).
    
    Note: The dataset by default comes with a test size larger than the train size.
    For more info see

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

def preprocess(image, label, preprocess_fn):
    image = tf.image.resize(image, IMG_SIZE)
    image = preprocess_fn(image)
    label = tf.one_hot(label, 102)
    return image, label

# Example for ResNet50 preprocessing
train_dataset_resnet = train_dataset.map(lambda x, y: preprocess(x, y, resnet_preprocess)) \
                                    .shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

validation_dataset_resnet = validation_dataset.map(lambda x, y: preprocess(x, y, resnet_preprocess)) \
                                               .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset_resnet = test_dataset.map(lambda x, y: preprocess(x, y, resnet_preprocess)) \
                                  .batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False

model_resnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(102, activation='softmax')
])

model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("resnet_best.h5", save_best_only=True)
]

history_resnet = model_resnet.fit(
    train_dataset_resnet,
    validation_data=validation_dataset_resnet,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.0783 - loss: 4.7916

32/32 ━━━━━━━━━━━━━━━━━━━━ 379s 12s/step - accuracy: 0.0814 - loss: 4.7701 - val_accuracy: 0.4657 - val_loss: 2.5025
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.7107 - loss: 1.6450

32/32 ━━━━━━━━━━━━━━━━━━━━ 379s 12s/step - accuracy: 0.7126 - loss: 1.6377 - val_accuracy: 0.7333 - val_loss: 1.4597
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9398 - loss: 0.7333

32/32 ━━━━━━━━━━━━━━━━━━━━ 370s 12s/step - accuracy: 0.9401 - loss: 0.7301 - val_accuracy: 0.7863 - val_loss: 1.0887
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9824 - loss: 0.3467

32/32 ━━━━━━━━━━━━━━━━━━━━ 374s 11s/step - accuracy: 0.9824 - loss: 0.3465 - val_accuracy: 0.8284 - val_loss: 0.9415
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9984 - loss: 0.2089

...

32/32 ━━━━━━━━━━━━━━━━━━━━ 384s 12s/step - accuracy: 0.9998 - loss: 0.1452 - val_accuracy: 0.8529 - val_loss: 0.7788
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 1.0000 - loss: 0.1108

32/32 ━━━━━━━━━━━━━━━━━━━━ 362s 11s/step - accuracy: 1.0000 - loss: 0.1106 - val_accuracy: 0.8500 - val_loss: 0.7435
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 1.0000 - loss: 0.0894

32/32 ━━━━━━━━━━━━━━━━━━━━ 368s 12s/step - accuracy: 1.0000 - loss: 0.0892 - val_accuracy: 0.8637 - val_loss: 0.7089
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 1.0000 - loss: 0.0644

32/32 ━━━━━━━━━━━━━━━━━━━━ 364s 11s/step - accuracy: 1.0000 - loss: 0.0644 - val_accuracy: 0.8588 - val_loss: 0.6876
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 1.0000 - loss: 0.0507

32/32 ━━━━━━━━━━━━━━━━━━━━ 364s 11s/step - accuracy: 1.0000 - loss: 0.0507 - val_accuracy: 0.8608 - val_loss: 0.6702


In [ ]:
loss, accuracy = model_resnet.evaluate(test_dataset_resnet)
print(f"ResNet50 Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


193/193 ━━━━━━━━━━━━━━━━━━━━ 1119s 6s/step - accuracy: 0.8249 - loss: 0.7963
ResNet50 Test Loss: 0.7834, Test Accuracy: 0.8296


#***VGGNET***

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess

# Image size for VGG16
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Preprocessing function
def preprocess_vgg(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = vgg_preprocess(image)  # VGG16 preprocessing
    label = tf.one_hot(label, 102)
    return image, label

# Preprocess datasets
train_dataset_vgg = train_dataset.map(preprocess_vgg).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
validation_dataset_vgg = validation_dataset.map(preprocess_vgg).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset_vgg = test_dataset.map(preprocess_vgg).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Load pre-trained VGG16 base model (exclude top)
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_model_vgg.trainable = False  # Freeze base layers

# Add custom classification head
model_vgg = models.Sequential([
    base_model_vgg,
    layers.GlobalAveragePooling2D(),
    layers.Dense(102, activation='softmax')
])

# Compile the model
model_vgg.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("vgg16_best.h5", save_best_only=True)
]

# Train the model
history_vgg = model_vgg.fit(
    train_dataset_vgg,
    validation_data=validation_dataset_vgg,
    epochs=5,
    callbacks=callbacks
)

# Evaluate on test dataset
loss, accuracy = model_vgg.evaluate(test_dataset_vgg)
print(f"VGG16 Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.0172 - loss: 12.3024 

32/32 ━━━━━━━━━━━━━━━━━━━━ 1253s 39s/step - accuracy: 0.0171 - loss: 12.2610 - val_accuracy: 0.0490 - val_loss: 7.3552
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.0649 - loss: 6.5202 

32/32 ━━━━━━━━━━━━━━━━━━━━ 1230s 39s/step - accuracy: 0.0661 - loss: 6.5045 - val_accuracy: 0.1402 - val_loss: 4.8080
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.2621 - loss: 3.7398 

32/32 ━━━━━━━━━━━━━━━━━━━━ 1222s 39s/step - accuracy: 0.2630 - loss: 3.7294 - val_accuracy: 0.2980 - val_loss: 3.4098
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.4745 - loss: 2.1705 

32/32 ━━━━━━━━━━━━━━━━━━━━ 1231s 39s/step - accuracy: 0.4755 - loss: 2.1636 - val_accuracy: 0.4000 - val_loss: 2.6559
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - accuracy: 0.7092 - loss: 1.1157 

32/32 ━━━━━━━━━━━━━━━━━━━━ 1230s 39s/step - accuracy: 0.7093 - loss: 1.1157 - val_accuracy: 0.4804 - val_loss: 2.2305
193/193 ━━━━━━━━━━━━━━━━━━━━ 3650s 19s/step - accuracy: 0.4241 - loss: 2.6259
VGG16 Test Loss: 2.5898, Test Accuracy: 0.4214
